# Satellite Imagery Data download Guide

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc">
<ul class="toc-item">
<li><span><a href="#Pre-requisites" data-toc-modified-id="Pre-requisites-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Pre-requisites</a></span></li>
<li><span><a href="#Instructions" data-toc-modified-id="Instructions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Instructions</a></span></li>
<li><span><a href="#Imports-and-Constants" data-toc-modified-id="Imports-and-Constants-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Imports and Constants</a></span></li>
<li><span><a href="#Constants" data-toc-modified-id="Constants-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Constants</a></span></li>
<li><span><a href="#Export-Images" data-toc-modified-id="Export-Images-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Export Images</a></span></li>
<li><span><a href="#Visualize-an-exported-image" data-toc-modified-id="Visualize-an-exported-image-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Visualize an exported image</a></span></li>
</ul>
</div>

In [0]:
!pip install --upgrade pip -qq
!pip install earthengine-api --upgrade -qq
!pip install geemap -qq
!pip install google-cloud-storage --quiet
!pip install tqdm -qq
!pip install gsutil -qq
!pip install gcloud -qq


## Imports and Constants

In [0]:
#================================================================================#
#================================================================================#
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
from typing import Any, Dict, Optional, Tuple
from google.cloud import storage
import gzip, shutil, os, pathlib, math, ee, geemap
os.chdir('/dbfs/mnt/raw/DataScientistSDSData/TLSED_test/')
import ee_utils
from gcloud import storage
#print(os.getcwd())

#================================================================================#
# ========== ADAPT THESE PARAMETERS ==========
#================================================================================#


# To export to Google Cloud Storage (GCS)
EXPORT = 'gcs'
BUCKET = 'dhs_tfrecords_raw_v2'
PREFIX = "dhs_tfrecords_raw_v2/"
DHS_EXPORT_FOLDER = 'dhs_tfrecords_raw_v2' # export location parameters
# To export from GCS to Azure Data Lake
DEST_DIR = "/dbfs/mnt/raw/DataScientistSDSData/TLSED_test/raw"

# input data paths
DHS_CSV_PATH = '/dbfs/mnt/raw/DataScientistSDSData/TLSED/dhs_clusters.csv'
# band names
MS_BANDS = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'TEMP1']

# image export parameters
PROJECTION = 'EPSG:3857'  # see https://epsg.io/3857
SCALE = 30                # export resolution: 30m/px
EXPORT_TILE_RADIUS = 127  # image dimension = (2*EXPORT_TILE_RADIUS) + 1 = 255px
CHUNK_SIZE = 1         # set to a small number (<= 50) if Google Earth Engine reports memory errors

#================================================================================
#================================================================================
#
# https://medium.com/@yvanjaquino/earth-engine-authentication-in-google-cloud-python-edition-6df832874f62
# Google Cloud console: https://console.cloud.google.com/welcome?project=ee-margheritaleonelli
# I created a default Earth Engine project and a service account for that (ee-transfer-learning@rare-deployment-461721-a2.iam.gserviceaccount.com)
# in order to authenticate in non-interactive way

service_account = 'ee-transfer-learning@rare-deployment-461721-a2.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, '/dbfs/mnt/raw/DataScientistSDSData/transfer_learning_rare-deployment-461721-a2-0ba3a6ab5a71.json')

ee.Initialize(credentials)
print('Earth Engine authenticated!')

Earth Engine authenticated!


In [0]:
def export_images(
        df: pd.DataFrame,
        country: str,
        year: int,
        export_folder: str,
        chunk_size: Optional[int] = None,
        ) -> Dict[Tuple[Any], ee.batch.Task]:
    '''
    Args
    - df: pd.DataFrame, contains columns ['lat', 'lon', 'country', 'year']
    - country: str, together with `year` determines the survey to export
    - year: int, together with `country` determines the survey to export
    - export_folder: str, name of folder for export
    - chunk_size: int, optionally set a limit to the # of images exported per TFRecord file
        - set to a small number (<= 50) if Google Earth Engine reports memory errors

    Returns: dict, maps task name tuple (export_folder, country, year, chunk) to ee.batch.Task
    '''
    subset_df = df[(df['country'] == country) & (df['year'] == year)].reset_index(drop=True)
    if chunk_size is None:
        num_chunks = 1
    else:
        num_chunks = int(math.ceil(len(subset_df) / chunk_size))
    tasks = {}

    for i in range(num_chunks):
        chunk_slice = slice(i * chunk_size, (i+1) * chunk_size - 1)  # df.loc[] is inclusive
        fc = ee_utils.df_to_fc(subset_df.loc[chunk_slice, :])
        
        try:
            start_date, end_date = ee_utils.surveyyear_to_range(year, nl=True)
        except ValueError as e:
            print(f"Skipping {country}, {year}: {e}")
            continue
        #start_date, end_date = ee_utils.surveyyear_to_range(year)

        # create 3-year Landsat composite image
        roi = fc.geometry()
        imgcol = ee_utils.LandsatSR(roi, start_date=start_date, end_date=end_date).merged
        imgcol = imgcol.map(ee_utils.mask_qaclear).select(MS_BANDS)
        img = imgcol.median()

        # add nightlights, latitude, and longitude bands
        img = ee_utils.add_latlon(img)
        img = img.addBands(ee_utils.composite_nl(year))

        fname = f'{country}_{year}_{i:02d}'
        tasks[(export_folder, country, year, i)] = ee_utils.get_array_patches(
            img=img, scale=SCALE, 
            ksize=EXPORT_TILE_RADIUS,
            points=fc, 
            export=EXPORT,
            prefix=export_folder, 
            fname=fname,
            bucket=BUCKET)
    return tasks

In [0]:
# Load surveys
dhs_df = pd.read_csv(DHS_CSV_PATH, float_precision='high', index_col=False).head(1000)
#dhs_df = dhs_df.head(10000)
dhs_surveys = list(dhs_df.groupby(['country', 'year']).groups.keys())

start_time = time.time()

for country, year in dhs_surveys:
    print(f"Starting export for {country}, {year}...")
    tasks = {}  # New empty task dict for each survey

    # Export images (submit export tasks)
    new_tasks = export_images(
        df=dhs_df, 
        country=country, 
        year=year,
        export_folder=DHS_EXPORT_FOLDER,
        chunk_size=CHUNK_SIZE)
    tasks.update(new_tasks)

    # Wait for all export tasks for this survey to finish
    print(f"Waiting on export tasks for {country}, {year}...")
    ee_utils.wait_on_tasks(tasks, poll_interval=60)
    print(f"Finished export for {country}, {year}.")
            
    # Give it some time to avoid overloading Google Earth Engine’s API (quota is being throttled (HTTP 503 = Service Unavailable)).
    time.sleep(100)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Total time taken: {elapsed_time:.2f} seconds ({elapsed_time/60:.2f} minutes)")

Starting export for angola, 2006...
Waiting on export tasks for angola, 2006...


  0%|          | 0/62 [00:00<?, ?it/s]

Task ('dhs_tfrecords_raw_v2', 'angola', 2006, 0) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2006, 1) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2006, 2) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2006, 3) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2006, 4) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2006, 5) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2006, 6) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2006, 7) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2006, 8) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2006, 9) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2006, 10) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords

  0%|          | 0/53 [00:00<?, ?it/s]

Task ('dhs_tfrecords_raw_v2', 'angola', 2007, 0) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2007, 1) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2007, 2) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2007, 3) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2007, 4) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2007, 5) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2007, 6) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2007, 8) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2007, 9) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2007, 10) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2007, 11) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecord

...

Task ('dhs_tfrecords_raw_v2', 'angola', 2015, 0) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2015, 1) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2015, 2) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2015, 3) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2015, 4) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2015, 6) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2015, 7) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2015, 8) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2015, 9) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2015, 10) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2015, 11) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecord

  0%|          | 0/308 [00:00<?, ?it/s]

Task ('dhs_tfrecords_raw_v2', 'angola', 2016, 0) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2016, 1) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2016, 2) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2016, 3) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2016, 4) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2016, 5) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2016, 6) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2016, 7) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2016, 8) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2016, 9) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw_v2', 'angola', 2016, 10) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords

0it [00:00, ?it/s]

Finished export for benin, 1996.
Total time taken: 2617.15 seconds (43.62 minutes)


In [0]:
# ---------------------- Download all files in the bucket ---------------------
SERVICE_ACCT_JSON = (
    "/dbfs/mnt/raw/DataScientistSDSData/transfer_learning_rare-deployment-461721-a2-0ba3a6ab5a71.json"
)

os.makedirs(DEST_DIR, exist_ok=True)

client = storage.Client.from_service_account_json(SERVICE_ACCT_JSON)

def download_blob_if_missing(blob):
    rel_path = blob.name[len(PREFIX):]
    local_path = os.path.join(DEST_DIR, rel_path)
    pathlib.Path(local_path).parent.mkdir(parents=True, exist_ok=True)
    if not os.path.exists(local_path):
        blob.download_to_filename(local_path)
        return True  # Downloaded
    else:
        print(f"File exists: {local_path}, skipping.")
        return False  # Skipped

blobs = [blob for blob in client.list_blobs(BUCKET, prefix=PREFIX) if blob.name.endswith(".gz")]
print(f"Total files to check: {len(blobs)}")
for blob in tqdm(blobs, desc="Downloading blobs"):
    download_blob_if_missing(blob)

downloaded_count = 0
skipped_count = 0
for blob in tqdm(blobs, desc="Downloading blobs"):
    if download_blob_if_missing(blob):
        downloaded_count += 1
    else:
        skipped_count += 1

print(f"Downloaded: {downloaded_count}, Skipped: {skipped_count}")

Total files to check: 969


File exists: /dbfs/mnt/raw/DataScientistSDSData/TLSED_test/raw/angola_2006_00.tfrecord.gz, skipping.


File exists: /dbfs/mnt/raw/DataScientistSDSData/TLSED_test/raw/angola_2006_01.tfrecord.gz, skipping.


File exists: /dbfs/mnt/raw/DataScientistSDSData/TLSED_test/raw/angola_2006_02.tfrecord.gz, skipping.


File exists: /dbfs/mnt/raw/DataScientistSDSData/TLSED_test/raw/angola_2006_03.tfrecord.gz, skipping.


...

File exists: /dbfs/mnt/raw/DataScientistSDSData/TLSED_test/raw/angola_2016_84.tfrecord.gz, skipping.
File exists: /dbfs/mnt/raw/DataScientistSDSData/TLSED_test/raw/angola_2016_85.tfrecord.gz, skipping.
File exists: /dbfs/mnt/raw/DataScientistSDSData/TLSED_test/raw/angola_2016_86.tfrecord.gz, skipping.
File exists: /dbfs/mnt/raw/DataScientistSDSData/TLSED_test/raw/angola_2016_87.tfrecord.gz, skipping.


File exists: /dbfs/mnt/raw/DataScientistSDSData/TLSED_test/raw/angola_2016_88.tfrecord.gz, skipping.
File exists: /dbfs/mnt/raw/DataScientistSDSData/TLSED_test/raw/angola_2016_89.tfrecord.gz, skipping.


File exists: /dbfs/mnt/raw/DataScientistSDSData/TLSED_test/raw/angola_2016_90.tfrecord.gz, skipping.
File exists: /dbfs/mnt/raw/DataScientistSDSData/TLSED_test/raw/angola_2016_91.tfrecord.gz, skipping.
File exists: /dbfs/mnt/raw/DataScientistSDSData/TLSED_test/raw/angola_2016_92.tfrecord.gz, skipping.


File exists: /dbfs/mnt/raw/DataScientistSDSData/TLSED_test/raw/angola_2016_93.tfrecord.gz, skipping.
File exists: /dbfs/mnt/raw/DataScientistSDSData/TLSED_test/raw/angola_2016_94.tfrecord.gz, skipping.
File exists: /dbfs/mnt/raw/DataScientistSDSData/TLSED_test/raw/angola_2016_95.tfrecord.gz, skipping.
File exists: /dbfs/mnt/raw/DataScientistSDSData/TLSED_test/raw/angola_2016_96.tfrecord.gz, skipping.


File exists: /dbfs/mnt/raw/DataScientistSDSData/TLSED_test/raw/angola_2016_97.tfrecord.gz, skipping.
File exists: /dbfs/mnt/raw/DataScientistSDSData/TLSED_test/raw/angola_2016_98.tfrecord.gz, skipping.
File exists: /dbfs/mnt/raw/DataScientistSDSData/TLSED_test/raw/angola_2016_99.tfrecord.gz, skipping.
Downloaded: 0, Skipped: 969
